In [2]:
import requests
import pandas as pd

# Read the API key from the file
with open('census-data-api-key.txt', 'r') as file:
    api_key = file.read().strip()

# Construct the API query URL for PUMS data for Utah
#url = f'https://api.census.gov/data/2019/pums?get=SERIALNO,NP,BDSP&for=state:49&key={api_key}'
url = f'https://api.census.gov/data/2022/acs/acs1/pums?get=SERIALNO,BDSP,HINCP,BLD&for=state:49&key={api_key}'

# Make the API request
response = requests.get(url)

# Check the response status and content
if response.status_code == 200:
    # Convert the response to a Pandas DataFrame
    data = response.json()
    columns = data[0]
    rows = data[1:]
    pums_data = pd.DataFrame(rows, columns=columns)
    
    # Convert columns to appropriate data types
    pums_data['BDSP'] = pd.to_numeric(pums_data['BDSP'])
    pums_data['HINCP'] = pd.to_numeric(pums_data['HINCP'])
    pums_data['BLD'] = pd.to_numeric(pums_data['BLD'])
    
    # Display the first few rows of the downloaded data
    print(pums_data.head())
    
    # Filter the data to include only single-family and multi-family units
    single_family = pums_data[pums_data['BLD'].isin([2, 3])]
    multi_family = pums_data[pums_data['BLD'].isin([4, 5, 6, 7, 8, 9])]

    # Analyze single-family data
    single_family_income = single_family.groupby('BDSP')['HINCP'].agg(['mean', 'count']).reset_index()
    single_family_income.columns = ['Number of Bedrooms', 'Average Household Income', 'Number of Records']

    # Analyze multi-family data
    multi_family_income = multi_family.groupby('BDSP')['HINCP'].agg(['mean', 'count']).reset_index()
    multi_family_income.columns = ['Number of Bedrooms', 'Average Household Income', 'Number of Records']

    # Display the results
    print(f"Single-Family Homes Income Breakdown: {len(single_family)} records")
    display(single_family_income)
    
    print(f"Multi-Family Homes Income Breakdown: {len(multi_family)} records")
    display(multi_family_income)

    # Optionally save the results to CSV files
    single_family_income.to_csv('single_family_income.csv', index=False)
    multi_family_income.to_csv('multi_family_income.csv', index=False)
else:
    print(f"Error: {response.status_code} - {response.content}")


        SERIALNO  BDSP  HINCP  BLD state
0  2022GQ0000032    -1 -60000    0    49
1  2022GQ0000163    -1 -60000    0    49
2  2022GQ0000198    -1 -60000    0    49
3  2022GQ0000320    -1 -60000    0    49
4  2022GQ0000483    -1 -60000    0    49
Single-Family Homes Income Breakdown: 9942 records


,Number of Bedrooms,Average Household Income,Number of Records
0,0,18543.207547,53
1,1,40493.893130,131
2,2,62468.928650,911
3,3,90070.356579,3040
4,4,121050.532663,2786
5,5,146099.904855,2039
6,6,173306.795506,890
7,8,252456.630435,92


Multi-Family Homes Income Breakdown: 1973 records


,Number of Bedrooms,Average Household Income,Number of Records
0,0,19243.304348,115
1,1,36956.635514,535
2,2,53427.186813,910
3,3,62583.394495,327
4,4,60810.869565,69
5,5,172685.000000,8
6,6,108666.666667,3
7,8,22116.666667,6
